In [1]:
%pip install azure-ai-ml
%pip install azure-identity

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from azure.ai.ml import MLClient
from azure.identity import AzureCliCredential
from azure.ai.ml.entities import ManagedOnlineEndpoint

credential = AzureCliCredential()

ml_client = MLClient(
    credential=credential,
    subscription_id="05c54f8a-eea2-4804-9157-82e9d39f3598",
    resource_group_name="wav2vec2-rg",
    workspace_name="wav2vec2-aml",
)

Class DeploymentTemplateOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [3]:
registry_ml_client = MLClient(credential=credential,registry_name="HuggingFace")

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [ ]:
models = registry_ml_client.models.list(name="facebook-hubert-base-ls960")

for m in models:
    print(m.name, m.version)


facebook-hubert-base-ls960 1


In [ ]:
model_name = "facebook-hubert-base-ls960"
model = registry_ml_client.models.get(name=model_name, version="1")
print(f"Model {model.name} version {model.version} retrieved successfully")

In [8]:
endpoint = ManagedOnlineEndpoint(
    name="hf-hubert2-endpoint-sti",
    auth_mode="key"
)

ml_client.online_endpoints.begin_create_or_update(endpoint).wait()
print("Endpoint created successfully")


Endpoint created successfully


In [9]:
from azure.ai.ml.entities import Environment

env = Environment(
    name="hf-wav2vec2-env",
    image="mcr.microsoft.com/azureml/curated/huggingface-transformers:latest",
)

In [ ]:
from azure.ai.ml.entities import ManagedOnlineDeployment

model_name = "facebook-hubert-base-ls960"
model = registry_ml_client.models.get(name=model_name, version="1")

demo_deployment = ManagedOnlineDeployment(
    name="hf-hubert-deployment-sti",
    endpoint_name="hf-hubert2-endpoint-sti",
    model=model,  # pass the model object directly for registry models
    instance_type="Standard_F2s_v2",
    instance_count=1,
)
ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
print("Deployment created successfully")
ml_client.online_endpoints.get(name="hf-hubert2-endpoint-sti")

Check: endpoint hf-wav2vec2-endpoint-sti exists


..

HttpResponseError: (ModelRegistryError) <UserError> The request is invalid.. ODataErrorDetails: Code: ModelDeploymentSettingsSkuBasedEngineNotFound. Message: No engine was found matching the requested sku type: Cpu. Make sure at least one engine in the list has the SkuArchitectureType property populated with the matching sku type..","Data":{"IsTerminal":true,"ErrorType":"ModelPresetNotFound"},"InnerException":{"ClassName":"System.AggregateException","Message":"One or more errors occurred.","Data":null,"InnerException":{"ClassName":"Microsoft.MachineLearning.InferenceDeployment.Http.HttpRequestWithStatusException","Message":"Failed to get model deployment settings. Status code: BadRequest. Error details: <UserError> The request is invalid.. ODataErrorDetails: Code: ModelDeploymentSettingsSkuBasedEngineNotFound. Message: No engine was found matching the requested sku type: Cpu. Make sure at least one engine in the list has the SkuArchitectureType property populated with the matching sku 

In [10]:
# Download the model locally
import os
from huggingface_hub import snapshot_download

# Create directory for the model
model_dir = "./hubert-model/model"
os.makedirs(model_dir, exist_ok=True)

# Download model from HuggingFace Hub
model_path = snapshot_download(
    repo_id="facebook/hubert-base-ls960",
    local_dir=model_dir,
    local_dir_use_symlinks=False
)

print(f"Model downloaded successfully to: {model_path}")

/home/shrey/Documents/azure-hubert/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/shrey/Documents/azure-hubert/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 6 files: 100%|██████████| 6/6 [02:35<00:00, 25.86s/it]

Model downloaded successfully to: /home/shrey/Documents/azure-hubert/hubert-model/model
